In [1]:
import pandas as pd
pd.set_option('display.max_rows', 500) 
pd.options.display.float_format = '{:,.2f}'.format
pd.options.display.float_format = lambda x: f'{x:,.1f}'.replace(',', 'X').replace('.', ',').replace('X', '.')
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

In [27]:
institutes = 'https://pollstergraph.com/ranking/institutes'
institutos = pd.read_html(institutes)
institutos = institutos[0]

In [3]:
codigo = [
    ['07630546000175','Datafolha'],
    ['19259002000128','AtlasIntel'],
    ['19090489000168','MDA'],
    ['81908345000140','Paraná Pesquisas'],
    ['REAL%20TIME','REAL TIME'],
    ['FUTURA','Futura'],
    ['13358105000169','6 Sigma'],
    ['10340949000194','Instituto Opinião (PR)'],
    ['11849437000110','Ipespe'],
    ['22445600000104','Quaest'],
    ['00654576000172','Veritá'],
    ['22675543000141','Estimativa'],
    ['RANKING','Ranking Pesquisa'],
    ['10956929000142','Datavox (PB)'],
    ['VOGA','Voga Pesquisas'],
    ['49875610000128','Brasmarket'],
    ['29550908000150','DataPoder360'],
    ['11077560000160','FSB Pesquisa'],
    ['GERP','GERP'],
    ['IDEIA','Ideia Big Data'],
    ['IPECIBOPE','Ipec (antigo Ibope)'],
    ['86849395000117','Sensus']
]
base = 'https://pollstergraph.com/institute/'

In [ ]:
#primeira heuristica que carrega os dados da tabela
for cod in codigo:
    tabelas = None
    print(cod)
    tabelas = pd.read_html(base+cod[0])
    tabela = tabelas[0]
    tabela[['porcentagem_1', 'candidato_1', 'porcentagem_2', 'candidato_2']] = (
        tabela['Resultado']
        .str.extract(r'(\d+(?:[.,]\d+)?)%\s*(.+?)(\d+(?:[.,]\d+)?)%\s*(.+)')
    )
    tabela['Erro Pesquisa'] = tabela['Erro Pesquisa'].astype(int)/10
    with pd.ExcelWriter('arquivos\\dados.xlsx', engine='openpyxl', mode='a') as writer:  # mode='a' para append
        print('Escrevendo')
        tabela.to_excel(writer, sheet_name=cod[1], index=False)
print('feito')

In [20]:
#iniciar selenium
options = webdriver.ChromeOptions()
# options.add_argument("--headless")

driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=options
)

wait = WebDriverWait(driver, 10)

inicio_total = time.time()
for cod in codigo:
    df_final = pd.DataFrame()
    start_time = time.time()
    tabelas = None
    print(cod)
    driver.get(base+cod[0])
    linhas = wait.until(
        EC.presence_of_all_elements_located((By.XPATH, ".//table/tbody/tr"))
    )
    i=0
    for linhaTabela in linhas: 
        cargo = linhaTabela.find_element(
            By.XPATH, ".//td[2]"
        ).text
        regiao = linhaTabela.find_element(
            By.XPATH, ".//td[3]"
        ).text
        turno = linhaTabela.find_element(
            By.XPATH, ".//td[5]"
        ).text
        linhaTabela.click()
    
        card = wait.until(
            EC.presence_of_element_located(
                (
                    By.CSS_SELECTOR,
                    "div.flex.h-dvh.w-dvw.flex-col.bg-white.font-ciutadella"
                )
            )
        )
        data = card.find_element(
            By.XPATH, ".//div[@class='grid grid-cols-3 gap-2.5 p-4 lg:p-7']/div[1]//div[2]"
        ).text
        amostra = card.find_element(
            By.XPATH, ".//div[@class='grid grid-cols-3 gap-2.5 p-4 lg:p-7']/div[2]//div[2]"
        ).text
        metodo = card.find_element(
            By.XPATH, ".//div[@class='grid grid-cols-3 gap-2.5 p-4 lg:p-7']/div[3]//div[2]"
        ).text
        margemErro = card.find_element(
            By.XPATH, ".//div[@class='grid grid-cols-3 gap-2.5 p-4 lg:p-7']/div[4]//div[2]"
        ).text
        erroPesquisa = card.find_element(
            By.XPATH, ".//div[@class='grid grid-cols-3 gap-2.5 p-4 lg:p-7']/div[5]//div[2]"
        ).text
        acerto = card.find_element(
            By.XPATH, ".//div[@class='grid grid-cols-3 gap-2.5 p-4 lg:p-7']/div[6]//div[2]"
        ).text
        candidatos = card.find_element(
            By.XPATH,
            ".//div[contains(@class,'c-scroll')]//table/tbody"
        )
        linhas = candidatos.find_elements(By.TAG_NAME, "tr")
        candidatos = []
        for linha in linhas:
            nome = linha.find_element(By.XPATH, ".//td[1]/div//div[2]").text
            try:
                percentual = linha.find_element(By.XPATH, ".//td[2]/div/span").text
            except:
                percentual = '0 ou Nulo'
                
            candidatos.append([i,cargo, regiao, turno, data, amostra, metodo, margemErro, erroPesquisa, acerto, nome, percentual])
            
        # cardDados = pd.DataFrame(candidatos, columns=['Cargo', 'Região', 'Turno', 'Data','Amostra', 'Metodo','Margem de Erro','Erro Pesquisa', 'Acertou', 'Nome', 'Percentual'])
        # info_card = cardDados.iloc[0][['Cargo', 'Região', 'Turno', "Data", "Amostra", "Metodo", "Margem de Erro", "Erro Pesquisa", "Acertou"]].to_dict()
        # for i, row in enumerate(cardDados.itertuples(index=False), start=1):
        #     info_card[f"nome_{i}"] = row.Nome
        #     info_card[f"percentual_{i}"] = row.Percentual
        #fechar card
        fechar = card.find_elements(By.XPATH, ".//div[@class='relative flex flex-col p-4 text-xs lg:p-7 lg:pb-0']/button")
        fechar[0].click()

        
        df_final = pd.concat([df_final, pd.DataFrame(candidatos)], ignore_index=True)
        i = i+1
    with pd.ExcelWriter('arquivos\\dados3.xlsx', engine='openpyxl', mode='a') as writer:  # mode='a' para append
        end_time = time.time()
        print(f"Loop para {cod[1]} demorou {end_time - start_time:.2f} segundos")
        df_final.to_excel(writer, sheet_name=cod[1], index=False)
fim_total = time.time()
print(f"Tempo total do loop: {fim_total - inicio_total:.2f} segundos")

['07630546000175', 'Datafolha']
Loop para Datafolha demorou 233.68 segundos
['19259002000128', 'AtlasIntel']
Loop para AtlasIntel demorou 125.57 segundos
['19090489000168', 'MDA']
Loop para MDA demorou 22.22 segundos
['81908345000140', 'Paraná Pesquisas']
Loop para Paraná Pesquisas demorou 267.79 segundos
['REAL%20TIME', 'REAL TIME']
Loop para REAL TIME demorou 321.38 segundos
['FUTURA', 'Futura']
Loop para Futura demorou 106.46 segundos
['13358105000169', '6 Sigma']
Loop para 6 Sigma demorou 11.39 segundos
['10340949000194', 'Instituto Opinião (PR)']
Loop para Instituto Opinião (PR) demorou 23.78 segundos
['11849437000110', 'Ipespe']
Loop para Ipespe demorou 86.56 segundos
['22445600000104', 'Quaest']
Loop para Quaest demorou 163.76 segundos
['00654576000172', 'Veritá']
Loop para Veritá demorou 152.59 segundos
['22675543000141', 'Estimativa']
Loop para Estimativa demorou 14.89 segundos
['RANKING', 'Ranking Pesquisa']
Loop para Ranking Pesquisa demorou 66.61 segundos
['10956929000142',

In [21]:
pd.DataFrame(candidatos)

,0,1,2,3,4,5,6,7,8,9,10,11
0,10,Governador,PA,1º,28 ago 2014,1.000,Presencial,3.1 Pp,"1,7 p.p.",-,Simão Jatene,"37,9%"
1,10,Governador,PA,1º,28 ago 2014,1.000,Presencial,3.1 Pp,"1,7 p.p.",-,Helder Barbalho,"36,9%"
2,10,Governador,PA,1º,28 ago 2014,1.000,Presencial,3.1 Pp,"1,7 p.p.",-,Ze Carlos Do Pv,"1,1%"
3,10,Governador,PA,1º,28 ago 2014,1.000,Presencial,3.1 Pp,"1,7 p.p.",-,Marco Carrera,"1,0%"
4,10,Governador,PA,1º,28 ago 2014,1.000,Presencial,3.1 Pp,"1,7 p.p.",-,Marco Antonio,0 ou Nulo
5,10,Governador,PA,1º,28 ago 2014,1.000,Presencial,3.1 Pp,"1,7 p.p.",-,Elton Braga,0 ou Nulo
6,10,Governador,PA,1º,28 ago 2014,1.000,Presencial,3.1 Pp,"1,7 p.p.",-,Voto Branco/nulo,"23,1%"
